In [20]:
pip install google-play-scraper


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from google_play_scraper import Sort, reviews
import pandas as pd

app_package_name = 'com.hso.motorku'

# Get reviews (up to 1000 reviews)
review_list, _ = reviews(
    app_package_name,
    lang='en',  # Language of reviews
    country='us',  # Country
    sort=Sort.NEWEST,  # Sort by NEWEST or MOST_RELEVANT
    count=5000,  # Max 5000 in one call
    filter_score_with=None  # Can filter reviews by score (e.g., 5 for only 5-star reviews)
)

# Convert to pandas DataFrame
df = pd.DataFrame(review_list)

# Preview
print(df[['userName', 'score', 'content', 'at']].head())





              userName  score  \
0       Reyhan Rizaldi      1   
1           Sena Shein      1   
2  Abi Musa Al Mohajir      5   
3       Arnez zulfitri      1   
4              Haryadi      2   

                                             content                  at  
0  aplikasi sampah, fitur track stnk dan plat gak... 2025-05-11 11:09:53  
1  jelek bgt apk nya. katanya jenis service belum... 2025-05-02 21:10:04  
2                                        mantap jaya 2025-05-01 08:21:51  
3  parah, ga bisa booking. ga ada data service, s... 2025-05-01 07:37:31  
4  Informasi riwayat servis kurang lengkap, tidak... 2025-04-30 17:58:10  


In [23]:
# Save to CSV
df.to_csv('motorkux_playstore_reviews.csv', index=False)
import os
print("File akan disimpan di:", os.getcwd())

# Save to Excel
df.to_excel('motorkux_playstore_reviews.xlsx', index=False)
import os
print("File akan disimpan di:", os.getcwd())

File akan disimpan di: c:\Users\X1 PRO\AppData\Local\Programs\Microsoft VS Code
File akan disimpan di: c:\Users\X1 PRO\AppData\Local\Programs\Microsoft VS Code


In [27]:
import requests
import pandas as pd
from time import sleep

app_id = '1500711969'  # ID aplikasi WeLoveHonda
country_codes = ['id', 'us', 'sg', 'my', 'ph', 'au', 'gb']  # daftar negara yang ingin di-scrape
all_reviews = []

for country in country_codes:
    print(f"Scraping reviews from country: {country}")
    page = 1
    while True:
        url = f"https://itunes.apple.com/{country}/rss/customerreviews/page={page}/id={app_id}/sortby=mostrecent/json"
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Request failed or no more pages for {country} page {page}.")
            break
        
        data = response.json()
        entries = data.get('feed', {}).get('entry', [])
        
        # Jika hanya ada metadata (1 entry) atau tidak ada, berhenti looping halaman
        if len(entries) <= 1:
            print(f"No more reviews on page {page} for {country}.")
            break
        
        # Proses review (skip metadata di index 0)
        for entry in entries[1:]:
            all_reviews.append({
                'author': entry['author']['name']['label'],
                'rating': entry['im:rating']['label'],
                'title': entry['title']['label'],
                'review': entry['content']['label'],
                'updated': entry['updated']['label'],
                'version': entry.get('im:version', {}).get('label', ''),
                'country': country
            })
        
        print(f"  Page {page} from {country} done. Total reviews collected: {len(all_reviews)}")
        page += 1
        sleep(1)  # delay supaya gak terlalu cepat request ke server Apple

# Buat DataFrame dan simpan Excel
df = pd.DataFrame(all_reviews)
df.to_xlsx('motorkux_reviews_multiple_countries.xlsx', index=False, encoding='utf-8-sig')

print(f"\nSelesai! Total reviews collected: {len(all_reviews)}")
print("Disimpan ke 'motorkux_reviews_multiple_countries.xlsx'")


Scraping reviews from country: id
  Page 1 from id done. Total reviews collected: 49
  Page 2 from id done. Total reviews collected: 98
  Page 3 from id done. Total reviews collected: 147
  Page 4 from id done. Total reviews collected: 196
  Page 5 from id done. Total reviews collected: 245
  Page 6 from id done. Total reviews collected: 294
  Page 7 from id done. Total reviews collected: 343
  Page 8 from id done. Total reviews collected: 392
  Page 9 from id done. Total reviews collected: 441
  Page 10 from id done. Total reviews collected: 490
Request failed or no more pages for id page 11.
Scraping reviews from country: us
No more reviews on page 1 for us.
Scraping reviews from country: sg
No more reviews on page 1 for sg.
Scraping reviews from country: my
No more reviews on page 1 for my.
Scraping reviews from country: ph
No more reviews on page 1 for ph.
Scraping reviews from country: au
No more reviews on page 1 for au.
Scraping reviews from country: gb
No more reviews on page 1

AttributeError: 'DataFrame' object has no attribute 'to_xlsx'